In [2]:
import numpy as np


d = np.array([
    [0, 10, 12, 11, 14],
    [10, 0, 13, 15, 8],
    [12, 13, 0, 9, 14],
    [11, 15, 9, 0, 16],
    [14, 8, 14, 16, 0]
])

iteration = 100
n_ants = 5
n_cities = 5


alpha = 1
beta = 2
evaporation = 0.5


visibility = np.zeros_like(d, dtype=float)
with np.errstate(divide='ignore'):
    visibility = 1 / d
visibility[np.isinf(visibility)] = 0
np.fill_diagonal(visibility, 0)


pheromone = 0.1 * np.ones((n_cities, n_cities))


best_route = None
best_cost = np.inf

for iteration_i in range(iteration):
    routes = np.zeros((n_ants, n_cities + 1), dtype=int)

    for ant in range(n_ants):
        visited = set()
        current_city = 0
        routes[ant, 0] = current_city
        visited.add(current_city)

        for step in range(1, n_cities):

            prob_numerators = []
            candidates = []

            for city in range(n_cities):
                if city not in visited:
                    tau = pheromone[current_city, city] ** alpha
                    eta = visibility[current_city, city] ** beta
                    prob_numerators.append(tau * eta)
                    candidates.append(city)

            prob_numerators = np.array(prob_numerators)
            if prob_numerators.sum() == 0:

                next_city = np.random.choice(candidates)
            else:
                probabilities = prob_numerators / prob_numerators.sum()
                next_city = np.random.choice(candidates, p=probabilities)

            routes[ant, step] = next_city
            visited.add(next_city)
            current_city = next_city


        routes[ant, n_cities] = routes[ant, 0]


    costs = np.zeros(n_ants)
    for ant in range(n_ants):
        cost = 0
        for city_index in range(n_cities):
            cost += d[routes[ant, city_index], routes[ant, city_index + 1]]
        costs[ant] = cost


    min_cost_index = np.argmin(costs)
    if costs[min_cost_index] < best_cost:
        best_cost = costs[min_cost_index]
        best_route = routes[min_cost_index].copy()


    pheromone = (1 - evaporation) * pheromone


    for ant in range(n_ants):
        contribution = 1.0 / costs[ant]
        for city_index in range(n_cities):
            i = routes[ant, city_index]
            j = routes[ant, city_index + 1]
            pheromone[i, j] += contribution
            pheromone[j, i] += contribution


print("Best route found:", best_route)
print("Cost of best route:", best_cost)

Best route found: [0 1 4 2 3 0]
Cost of best route: 52.0
